In [1]:
import torch
import torch.nn as nn
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import chess
from nnue import HIDDEN_SIZE, INPUT_SIZE, EVAL_SCALE, fen_to_onehot, NNUE
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
#df = pd.read_csv("./train.csv")

In [3]:
#df['short fen'] = df['FEN'].map(lambda s : s.split(" ")[0])
#df['score'] = df['Evaluation'].map(lambda x : 0.5 + min(1000, max(-1000, x)) / 2000)

In [4]:
#BATCH_SIZE = 256
#testing_df = df.sample(frac=0.01)
#test_dataset = np.array_split(testing_df[['short fen', 'score']], int(len(testing_df) / BATCH_SIZE))

In [2]:
class TrainingNNUE(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.l1 = nn.Linear(INPUT_SIZE, HIDDEN_SIZE, bias=True)
        self.l2 = nn.Linear(2 * HIDDEN_SIZE, 1, bias = True)

    def forward(self, x):
        x = self.l1(x)      # (B, H)
        x = torch.concat((x, -x), dim=-1)   # (B, 2*H)
        x = torch.square(torch.clip(x, 0, 1))
        x = self.l2(x)
        return torch.sigmoid(x)


In [3]:
model = TrainingNNUE()
model.load_state_dict(torch.load("./models/nnue1.ckpt", weights_only  = True))

<All keys matched successfully>

In [ ]:
#torch.onnx.export(model,(torch.randn((768,)),) , dynamo = True, f='./models/nnue1.onnx')

[torch.onnx] Obtain model graph for `TrainingNNUE([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `TrainingNNUE([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 1 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.0+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"x"<FLOAT,[768]>
            ),
            outputs=(
                %"sigmoid"<FLOAT,[1]>
            ),
            initializers=(
                %"l1.bias"<FLOAT,[256]>{TorchTensor(...)},
                %"l2.bias"<FLOAT,[1]>{TorchTensor<FLOAT,[1]>(Parameter containing: tensor([-0.0477], requires_grad=True), name='l2.bias')},
                %"val_0"<FLOAT,[768,256]>{Tensor(...)},
                %"val_6"<FLOAT,[512,1]>{Tensor(...)},
                %"concat_min"<FLOAT,[]>{Tensor<FLOAT,[]>(array(0., dtype=float32), name='concat_min')},
                %"concat_max"<FLOAT,[]>{Tensor<FLOAT,[]>(array(1., dtype=float32), name='concat_max')},
                %"val_

In [ ]:
model.cpu()
nnue = NNUE()

nnue.params['acc_weights'] = model.l1.weight.squeeze().detach().numpy()
nnue.params['acc_biases'] = model.l1.bias.squeeze().detach().numpy()
nnue.params['output_weights'] = model.l2.weight.squeeze().detach().numpy()
nnue.params['output_bias'] = model.l2.bias.squeeze().detach().numpy()

board = chess.Board()
board.set_board_fen()
m1 = fen_to_onehot(board.board_fen())
nnue.first_forward(m1)

board.push(next(board.generate_legal_moves()))
m2 = fen_to_onehot(board.board_fen())

s = time.time()
for _ in range(1000) : nnue.first_forward(fen_to_onehot(board.board_fen()))
print(time.time() - s)

s = time.time()
for _ in range(1000) : nnue.forward(fen_to_onehot(board.board_fen()))
print(time.time() - s)

0.38837289810180664
0.5725052356719971


In [ ]:
board.set_board_fen("7Q/p1p1kppp/4b1q1/8/8/2NP4/PPP2PPP/R1B1R1K1")
board.turn = not board.turn
m1 = fen_to_onehot(board.board_fen())
m2 = fen_to_onehot(board.mirror().board_fen())
nnue.first_forward(m2), nnue.first_forward(m1)

(tensor(-3.4146), tensor(3.3894))

: 